In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer



#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
df_flight = pd.read_csv('../Files/Customer Flight Activity.csv')
df_loyalty = pd.read_csv('../Files/Customer Loyalty History.csv')

In [3]:
df_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [4]:
df_flight.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,405624.0,550037.873084,258935.286969,100018.0,326961.00,550834.0,772194.00,999986.0
Year,405624.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.115052,5.225518,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.031805,2.076869,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.146858,6.521227,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.880059,1433.155320,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.692721,146.599831,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.696872,125.486049,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.0,0.00,0.0,0.00,71.0


In [5]:
df_flight.dtypes

Loyalty Number                   int64
Year                             int64
Month                            int64
Flights Booked                   int64
Flights with Companions          int64
Total Flights                    int64
Distance                         int64
Points Accumulated             float64
Points Redeemed                  int64
Dollar Cost Points Redeemed      int64
dtype: object

In [6]:
df_flight.isnull().sum()

Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

In [7]:
df_flight.duplicated().sum()

1864

In [8]:
cols = ['Country', 'Province', 'City',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month']

for column in cols:
    print(f"Frequência de valores na coluna '{column}':\n")
    print(df_loyalty[column].value_counts())

Frequência de valores na coluna 'Country':

Country
Canada    16737
Name: count, dtype: int64
Frequência de valores na coluna 'Province':

Province
Ontario                 5404
British Columbia        4409
Quebec                  3300
Alberta                  969
Manitoba                 658
New Brunswick            636
Nova Scotia              518
Saskatchewan             409
Newfoundland             258
Yukon                    110
Prince Edward Island      66
Name: count, dtype: int64
Frequência de valores na coluna 'City':

City
Toronto           3351
Vancouver         2582
Montreal          2059
Winnipeg           658
Whistler           582
Halifax            518
Ottawa             509
Trenton            486
Edmonton           486
Quebec City        485
Dawson Creek       444
Fredericton        425
Regina             409
Kingston           401
Tremblant          398
Victoria           389
Hull               358
West Vancouver     324
St. John's         258
Thunder Bay        256
S

In [9]:
df_loyalty[df_loyalty['Cancellation Year'].notnull()]

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
19,354730,Canada,New Brunswick,Fredericton,E3B 2H2,Female,College,NaN,Married,Star,3885.46,Standard,2014,10,2018.0,1.0
31,201574,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,51375.0,Married,Star,3893.31,Standard,2015,4,2015.0,12.0
39,834891,Canada,British Columbia,Vancouver,V5R 1W3,Female,Bachelor,54460.0,Married,Star,3911.12,Standard,2017,3,2017.0,4.0
45,733338,Canada,Ontario,Thunder Bay,K8T 5M5,Female,Bachelor,80697.0,Married,Star,3919.37,Standard,2016,6,2017.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16706,409984,Canada,Yukon,Whitehorse,Y2K 6R0,Male,College,NaN,Single,Star,7452.23,2018 Promotion,2018,4,2018.0,12.0
16713,969719,Canada,British Columbia,West Vancouver,V6V 8Z3,Male,Bachelor,88356.0,Married,Star,40235.97,Standard,2016,5,2017.0,1.0
16718,795743,Canada,Ontario,Thunder Bay,K8T 5M5,Male,College,NaN,Married,Star,43019.01,Standard,2013,11,2014.0,7.0
16730,672917,Canada,Ontario,Toronto,M1R 4K3,Female,Bachelor,76778.0,Single,Star,58166.55,Standard,2015,1,2015.0,9.0


In [10]:
df_loyalty.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.00,326603.00,550434.00,772019.00,999986.00
Salary,12499.0,79245.609409,35008.297285,-58486.00,59246.50,73455.00,88517.50,407228.00
CLV,16737.0,7988.896536,6860.982280,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,16737.0,6.669116,3.398958,1.00,4.00,7.00,10.00,12.00
Cancellation Year,2067.0,2016.503145,1.380743,2013.00,2016.00,2017.00,2018.00,2018.00
Cancellation Month,2067.0,6.962748,3.455297,1.00,4.00,7.00,10.00,12.00


In [11]:
df_loyalty[df_loyalty['Salary'] <0]['Salary'].count() # ADDING COUNT TO SEE HOW MANY ARE AFTER SORTING


20

In [12]:
df_loyalty.duplicated().sum()

0

In [13]:
df_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

In [14]:
df_loyalty['Salary'].isnull().sum()/ df_loyalty.shape[0]*100

25.321144769074504

In [15]:
df_loyalty['Salary'][1]

nan

In [16]:
df_loyalty['Salary'][1].dtype

dtype('float64')

In [17]:
df_loyalty['Salary'] = pd.to_numeric(df_loyalty['Salary'], errors='coerce')
print(df_loyalty['Salary'].isna().sum())
print(df_loyalty[df_loyalty['Salary'].isna() | (df_loyalty['Salary'] == '')])

df_loyalty['Salary'] = df_loyalty['Salary'].replace('', np.nan)

print(df_loyalty[df_loyalty['Salary'].isna()]['Salary'].count())



4238
       Loyalty Number Country          Province       City Postal Code  \
1              549612  Canada           Alberta   Edmonton     T3G 6Y6   
2              429460  Canada  British Columbia  Vancouver     V6E 3D9   
3              608370  Canada           Ontario    Toronto     P1W 1K4   
6              927943  Canada           Ontario    Toronto     P5S 6R4   
13             988178  Canada            Quebec   Montreal     H4G 3T4   
...               ...     ...               ...        ...         ...   
16721          632951  Canada           Alberta   Edmonton     T9G 1W3   
16727          546773  Canada  British Columbia  Vancouver     V6E 3D9   
16731          900501  Canada           Ontario    Sudbury     M5V 1G5   
16732          823768  Canada  British Columbia  Vancouver     V6E 3Z3   
16734          776187  Canada  British Columbia  Vancouver     V5R 1W3   

       Gender Education  Salary Marital Status Loyalty Card       CLV  \
1        Male   College     NaN  

In [18]:

df_loyalty['Salary'].isnull().sum()

4238

In [19]:
nan_salary_count = df_loyalty[df_loyalty['Salary'].isnull()].shape[0]
print(nan_salary_count)

4238


In [22]:
df_loyalty[df_loyalty['Salary'].isnull()].sample(50)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
10398,490439,Canada,Ontario,Ottawa,K1G 4Z0,Male,College,NaN,Married,Star,2415.83,Standard,2017,5,2018.0,1.0
4174,142763,Canada,Ontario,Toronto,P2T 6G3,Male,College,NaN,Single,Nova,2747.77,Standard,2018,12,NaN,NaN
3719,572663,Canada,Quebec,Hull,J8Y 3Z5,Male,College,NaN,Single,Aurora,17253.73,Standard,2012,8,NaN,NaN
9996,393403,Canada,Quebec,Montreal,H2Y 2W2,Female,College,NaN,Married,Star,2236.09,Standard,2015,8,NaN,NaN
6224,210630,Canada,Quebec,Montreal,H2T 9K8,Female,College,NaN,Single,Nova,4922.17,Standard,2015,2,NaN,NaN
16721,632951,Canada,Alberta,Edmonton,T9G 1W3,Female,College,NaN,Married,Star,44771.30,Standard,2018,7,NaN,NaN
11120,575487,Canada,Ontario,Kingston,M9K 2P4,Male,College,NaN,Single,Star,3788.81,2018 Promotion,2018,2,NaN,NaN
5018,874984,Canada,British Columbia,Vancouver,V6E 3D9,Female,College,NaN,Single,Nova,3362.55,Standard,2015,7,NaN,NaN
14947,160320,Canada,Quebec,Montreal,H2T 2J6,Female,College,NaN,Single,Star,8150.57,Standard,2014,10,NaN,NaN
9617,464962,Canada,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,23706.11,Standard,2012,12,NaN,NaN
